In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_selection import chi2, SelectPercentile, SelectKBest

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
%%time
df = pd.read_pickle('../input/microsoft-malware-first-data-analsys/dfn.pickle')
y = df['HasDetections']
df.drop('HasDetections', axis=1, inplace=True)
print(df.shape)

# Need to get the training set as well, in order to process it in the same way
# Test set does not have the label
df_test = pd.read_csv('../input/microsoft-malware-prediction/test.csv', usecols=df.columns)

In [ ]:
# Redundant Check
assert all(df.columns == df_test.columns)

# Missing Data

In [ ]:
df.count().sort_values().plot.barh(figsize=(10, 10))
plt.show()

In [ ]:
## Disregard features that have less than half of all the data missing
low_count = df.iloc[:, df.count().values < df.shape[0]/2].columns
print('Features droped:\n {}'.format(low_count))
df.drop(low_count, axis=1, inplace=True, errors='ignore')
print(df.shape)

# Drop from the test set as well
df_test.drop(low_count, axis=1, inplace=True, errors='ignore')

# Remove Low Variance

In [ ]:
v = lambda p: p*(1-p)
p=0.95

# Remove Features with very low variance
to_drop = []
for c in df.columns:
    if df[c].var() < v(p):
        to_drop.append(c)
        
df.drop(to_drop, axis=1, inplace=True, errors='ignore')
df_test.drop(to_drop, axis=1, inplace=True, errors='ignore')

print("Training: {}".format(df.shape))
print("Testing: {}".format(df_test.shape))

In [ ]:
# Redundant Check
assert all(df.columns == df_test.columns)

# Manual Feature Processing

## Categorical
There are some identifiers that are categorical

### Identifiers

In [ ]:
categorical = [c for c in df.columns if "identifier" in c.lower()] + ['OsBuild', 'OsSuite']
df[categorical].astype("category").describe().transpose().sort_values(by="unique", ascending=False)

In [ ]:
# This columns were removed just by looking at their names, no other reason besides intuition
to_drop = ['CityIdentifier', 'Census_OSInstallLanguageIdentifier', 'LocaleEnglishNameIdentifier', 'GeoNameIdentifier',
          'IeVerIdentifier', 'Wdft_RegionIdentifier', 'Census_ProcessorManufacturerIdentifier', 'Census_OEMModelIdentifier',
          'Census_FirmwareVersionIdentifier', 'AVProductStatesIdentifier', 'Census_OEMNameIdentifier', 'Census_ProcessorModelIdentifier',
           'Census_FirmwareManufacturerIdentifier', 'Census_OSUILocaleIdentifier', 'Census_OSBuildNumber', 'Census_OSBuildRevision'
          ]
# Categories to make dummies out of
to_dummy = [c for c in categorical if c not in to_drop]

df.drop(to_drop, axis=1, inplace=True, errors='ignore')
df_test.drop(to_drop, axis=1, inplace=True, errors='ignore')

### AVProductsInstalled

In [ ]:
df['AVProductsInstalled'].value_counts().plot.barh()
plt.show()

to_dummy.append('AVProductsInstalled')

df['AVProductsInstalled'] = df['AVProductsInstalled'].apply(lambda x: 'None' if x == 0 else ('>2' if x > 1 else 1))
df_test['AVProductsInstalled'] = df_test['AVProductsInstalled'].apply(lambda x: 'None' if x == 0 else ('>2' if x > 1 else 1))

### RtpStateBitfield

In [ ]:
df['RtpStateBitfield'].value_counts().plot.barh()
plt.show()

# Drop this one because 
df.drop('RtpStateBitfield', axis=1, inplace=True)
df_test.drop('RtpStateBitfield', axis=1, inplace=True)

### Census_InternalPrimaryDiagonalDisplaySizeInInches

In [ ]:
df[['Census_InternalPrimaryDiagonalDisplaySizeInInches', 
    'Census_InternalPrimaryDisplayResolutionVertical',
    'Census_InternalPrimaryDisplayResolutionHorizontal'
   ]].hist(bins=100, figsize=(20, 15))
plt.show()

# Drop these rows (probably useless)
df.drop(['Census_InternalPrimaryDiagonalDisplaySizeInInches', 
    'Census_InternalPrimaryDisplayResolutionVertical',
    'Census_InternalPrimaryDisplayResolutionHorizontal'
   ], axis=1, inplace=True)

df_test.drop(['Census_InternalPrimaryDiagonalDisplaySizeInInches', 
    'Census_InternalPrimaryDisplayResolutionVertical',
    'Census_InternalPrimaryDisplayResolutionHorizontal'
   ], axis=1, inplace=True)

### Wdft_IsGamer

In [ ]:
df['Wdft_IsGamer'].value_counts().plot.barh()
plt.show()

### Census_ProcessorCoreCount

In [ ]:
df['Census_ProcessorCoreCount'].value_counts().plot.barh(figsize=(10, 10))
plt.show()

to_dummy.append('Census_ProcessorCoreCount')

# Group 
df['Census_ProcessorCoreCount'] = df['Census_ProcessorCoreCount'].apply(lambda x: 'other' if x == 1 else ('DualCore' if x == 2 else ("QuadCore" if x==4 else "other")))
df_test['Census_ProcessorCoreCount'] = df_test['Census_ProcessorCoreCount'].apply(lambda x: 'other' if x == 1 else ('DualCore' if x == 2 else ("QuadCore" if x==4 else "other")))

### Census_TotalPhysicalRAM

In [ ]:
# Convert to GB
df['Census_TotalPhysicalRAM'] = df['Census_TotalPhysicalRAM'].apply(lambda x: x/1024)
df_test['Census_TotalPhysicalRAM'] = df_test['Census_TotalPhysicalRAM'].apply(lambda x: x/1024)

# This is in bytes
df['Census_TotalPhysicalRAM'].hist(bins=100, figsize=(10, 10))
plt.show()

In [ ]:
ram = lambda x: "<1GB" if  x <= 1 else ("2GB" if x == 2 else ("4GB" if x == 4 else ("8GB" if x == 8 else "other")))
df['Census_TotalPhysicalRAM'] = df['Census_TotalPhysicalRAM'].apply(ram)
df_test['Census_TotalPhysicalRAM'] = df_test['Census_TotalPhysicalRAM'].apply(ram)

In [ ]:
# This is in bytes
to_dummy.append('Census_TotalPhysicalRAM')

df['Census_TotalPhysicalRAM'].value_counts().plot.barh()
plt.show()

### UacLuaenable

In [ ]:
df['UacLuaenable'].value_counts().plot.barh()
plt.show()

# Drop this
df.drop('UacLuaenable', axis=1, inplace=True)
df_test.drop('UacLuaenable', axis=1, inplace=True)

### Census_SystemVolumeTotalCapacity

In [ ]:
# Convert to GB
df['Census_SystemVolumeTotalCapacity'] = df['Census_SystemVolumeTotalCapacity'].apply(lambda x: x/1024)
df_test['Census_SystemVolumeTotalCapacity'] = df_test['Census_SystemVolumeTotalCapacity'].apply(lambda x: x/1024)

In [ ]:
df['Census_SystemVolumeTotalCapacity'].hist(bins=100)
plt.show()

In [ ]:
vol_cap = lambda x: "<120G" if x < 120 else ("240G" if x <= 240  else ("500GB" if x <= 500 else ("1TB" if x <= 1000 else "other")))
df['Census_SystemVolumeTotalCapacity'] = df['Census_SystemVolumeTotalCapacity'].apply(vol_cap)
df_test['Census_SystemVolumeTotalCapacity'] = df_test['Census_SystemVolumeTotalCapacity'].apply(vol_cap)
df['Census_SystemVolumeTotalCapacity'].value_counts().plot.barh()
plt.show()

to_dummy.append('Census_SystemVolumeTotalCapacity')

### Census_PrimaryDiskTotalCapacity

In [ ]:
df['Census_PrimaryDiskTotalCapacity'] = df['Census_PrimaryDiskTotalCapacity'].apply(lambda x: x/1024)
df_test['Census_PrimaryDiskTotalCapacity'] = df_test['Census_PrimaryDiskTotalCapacity'].apply(lambda x: x/1024)

vol_cap = lambda x: "<120G" if x < 120 else ("240G" if x <= 240  else ("500GB" if x <= 500 else ("1TB" if x <= 1000 else "other")))
df['Census_PrimaryDiskTotalCapacity'] = df['Census_PrimaryDiskTotalCapacity'].apply(vol_cap)
df_test['Census_PrimaryDiskTotalCapacity'] = df_test['Census_PrimaryDiskTotalCapacity'].apply(vol_cap)
df['Census_PrimaryDiskTotalCapacity'].value_counts().plot.barh()
plt.show()

to_dummy.append('Census_PrimaryDiskTotalCapacity')

## Build Dummies

In [ ]:
%%time
df = pd.get_dummies(df, columns=to_dummy)

# Remove Features with very low variance again (after the dummies)
to_drop = []
for c in df.columns:
    if df[c].var() < v(p):
        to_drop.append(c)
df.drop(to_drop, axis=1, inplace=True, errors='ignore')

df_test = pd.get_dummies(df_test, columns=to_dummy)
# Only keep columns in test that are on the training set
df_test = df_test[[c for c in df.columns if c in df_test.columns]]

assert all(df.columns == df_test.columns)

print("Training: {}".format(df.shape))
print("Testing: {}".format(df_test.shape))

In [ ]:
df.describe().transpose().sort_values(by='max')

# Chi2

In [ ]:
skb = SelectKBest(chi2, k=20)

# Only the rows that would not have any Nans
ind = df.dropna().index

# Fit 
skb.fit(df.loc[ind, :], y[ind])

# Get the columns in each set
df = df.iloc[:, skb.get_support()]
df_test = df_test.iloc[:, skb.get_support()]

# Save

In [ ]:
df.describe()

In [ ]:
assert all(df.columns == df_test.columns)

In [ ]:
df.to_pickle("dfn_train_processed.pickle")
df_test.to_pickle("dfn_test_processed.pickle")